In [1]:
print("hi")

hi


In [19]:
import requests

API_URL = "http://cs-gpu07.srv.aau.dk:8000/generate"

def query_flan(prompt):
    payload = {
        "inputs": prompt,
        "parameters": {"max_new_tokens": 200}
    }
    response = requests.post(API_URL, json=payload)
    return response.json()

print(query_flan("Translate this to Spanish: Hello, how are you?"))

{'generated_text': 'Hola, cómo estás?'}


In [40]:
print(query_flan("My idiot friend visited me in New York. So where does he live?"))

{'generated_text': 'in california'}
